In [27]:
import pandas as pd
import random as rnd
import json
import copy

In [28]:
#Load data
class_data = pd.read_csv('C:/Users/ADMIN/Downloads/Compressed/class.csv')
teacher_data = pd.read_csv('C:/Users/ADMIN/Downloads/Compressed/teacher.csv')
subject_data = pd.read_csv('C:/Users/ADMIN/Downloads/Compressed/subject.csv')
subject_list_data = pd.read_csv('C:/Users/ADMIN/Downloads/Compressed/subject_list.csv')
timeslot_data = pd.read_csv('C:/Users/ADMIN/Downloads/Compressed/timeslot.csv')
timeslot_list_data = pd.read_csv('C:/Users/ADMIN/Downloads/Compressed/timeslot_list.csv')
format_type_data = pd.read_csv('C:/Users/ADMIN/Downloads/Compressed/format_type.csv')

In [29]:
#Choosing grade to generate timetable
grade = int(input('Nhập khối: '))

#Filter data by grade
class_data = class_data[class_data['grade_id']==grade]
teacher_data = teacher_data[teacher_data['grade_id']==grade]
subject_data = subject_data[subject_data['grade_id']==grade]

Nhập khối: 11


In [30]:
class_data.head()

,id,class_name,grade_id,type_of_education_id,format_type_id,compulsory_subject_list,optional_subject_list
22,2843,11SN1,11,1,1,"[841, 843, 838, 852, 839, 840, 844, 853, 842, ...",[]
23,2844,11SN2,11,1,1,"[851, 832, 834, 835, 837, 838, 839, 840, 841, ...",[]
24,2845,11CT,11,2,1,"[851, 832, 834, 835, 837, 838, 839, 840, 841, ...",[]
25,2846,11A,11,3,1,"[851, 832, 834, 835, 837, 838, 839, 840, 841, ...",[]
26,2847,11A1,11,3,2,"[851, 832, 834, 835, 837, 838, 839, 840, 841, ...",[]


In [31]:
# Transform class data
class_data['optional_subject_list'] = class_data['optional_subject_list'].apply(
    lambda x: x.replace('[', '').replace(']', ''))
class_data['compulsory_subject_list'] = class_data['compulsory_subject_list'].apply(
    lambda x: x.replace('[', '').replace(']', ''))
class_data['subject_list'] = ""
for idx, row in class_data.iterrows():
    if row['optional_subject_list'] == '':
        class_data.at[idx, 'subject_list'] = "[" + class_data.at[idx, 'compulsory_subject_list'] + "]"
    else:
        class_data.at[idx, 'subject_list'] = "[" + class_data.at[idx, 'compulsory_subject_list'] + ", " + \
                                             class_data.at[idx, 'optional_subject_list'] + "]"
class_data = class_data.drop(['compulsory_subject_list', 'optional_subject_list'], axis=1)

In [32]:
class_data.head()

,id,class_name,grade_id,type_of_education_id,format_type_id,subject_list
22,2843,11SN1,11,1,1,"[841, 843, 838, 852, 839, 840, 844, 853, 842, ..."
23,2844,11SN2,11,1,1,"[851, 832, 834, 835, 837, 838, 839, 840, 841, ..."
24,2845,11CT,11,2,1,"[851, 832, 834, 835, 837, 838, 839, 840, 841, ..."
25,2846,11A,11,3,1,"[851, 832, 834, 835, 837, 838, 839, 840, 841, ..."
26,2847,11A1,11,3,2,"[851, 832, 834, 835, 837, 838, 839, 840, 841, ..."


In [33]:
format_type_data.head()

,id,rs_school_day_time_slot_list_id
0,1,"[1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15, 2..."
1,2,"[1, 2, 3, 4, 6, 7, 8, 9, 16, 17, 18, 19, 20, 2..."
2,3,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 1..."
3,4,"[1, 2, 3, 4, 5, 11, 12, 13, 14, 16, 17, 18, 19..."
4,5,"[1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 16, 1..."


In [34]:
timeslot_list_data.head()

,id,name
0,181,Tiết 1 sáng
1,182,Tiết 2 sáng
2,183,Tiết 3 sáng
3,184,Tiết 4 sáng
4,185,Tiết 5 sáng


In [35]:
timeslot_data.head()

,id,school_day_id,time_slot_id
0,1,1,181
1,2,1,182
2,3,1,183
3,4,1,184
4,5,1,185


In [36]:
# Transform format_type data
format_type_data['rs_school_day_time_slot_list_id'] = format_type_data.rs_school_day_time_slot_list_id.apply(
    lambda x: x[1:-1].split(','))
format_type_data = format_type_data.explode('rs_school_day_time_slot_list_id').astype('int64')
format_type_data = format_type_data.merge(timeslot_data, left_on='rs_school_day_time_slot_list_id', right_on='id')
format_type_data = format_type_data.drop(['id_y', 'rs_school_day_time_slot_list_id'], axis=1).rename(
    columns={'id_x': 'id'})
format_type_data = format_type_data.merge(timeslot_list_data, left_on='time_slot_id', right_on='id')
format_type_data['timeslot'] = list(
    zip(format_type_data.school_day_id, format_type_data.time_slot_id, format_type_data.name))
format_type_data = format_type_data.drop(['school_day_id', 'time_slot_id', 'id_y', 'name'], axis=1).rename(
    columns={'id_x': 'id'})
format_type_data = format_type_data.groupby('id').agg(lambda x: list(x))

In [37]:
format_type_data.head()

,timeslot
id,
1,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),..."
2,"[(1, 181, Tiết 1 sáng), (3, 181, Tiết 1 sáng),..."
3,"[(2, 181, Tiết 1 sáng), (4, 181, Tiết 1 sáng),..."
4,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),..."
5,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),..."


In [38]:
teacher_data.head()

,subject_id,teacher_id,grade_id
2,841,8836,11
3,840,8836,11
7,834,8863,11
10,853,8863,11
13,854,8863,11


In [39]:
# Assign teacher to create timetable
teacher_data_list = []
for i, row in teacher_data.iterrows():
    teacher = [row.subject_id, row.teacher_id, row.grade_id]
    teacher_data_list.append(teacher)

In [40]:
teacher_data_list

[[841, 8836, 11],
 [840, 8836, 11],
 [834, 8863, 11],
 [853, 8863, 11],
 [854, 8863, 11],
 [834, 8864, 11],
 [853, 8864, 11],
 [854, 8864, 11],
 [834, 8865, 11],
 [853, 8865, 11],
 [854, 8865, 11],
 [834, 8866, 11],
 [853, 8866, 11],
 [854, 8866, 11],
 [834, 8867, 11],
 [853, 8867, 11],
 [854, 8867, 11],
 [834, 8868, 11],
 [853, 8868, 11],
 [854, 8868, 11],
 [834, 8869, 11],
 [853, 8869, 11],
 [854, 8869, 11],
 [834, 8870, 11],
 [853, 8870, 11],
 [854, 8870, 11],
 [834, 8871, 11],
 [853, 8871, 11],
 [854, 8871, 11],
 [834, 8872, 11],
 [853, 8872, 11],
 [854, 8872, 11],
 [834, 8873, 11],
 [853, 8873, 11],
 [854, 8873, 11],
 [834, 8874, 11],
 [853, 8874, 11],
 [854, 8874, 11],
 [834, 8875, 11],
 [853, 8875, 11],
 [854, 8875, 11],
 [834, 8876, 11],
 [853, 8876, 11],
 [854, 8876, 11],
 [834, 8877, 11],
 [853, 8877, 11],
 [854, 8877, 11],
 [834, 8878, 11],
 [853, 8878, 11],
 [854, 8878, 11],
 [834, 8879, 11],
 [853, 8879, 11],
 [854, 8879, 11],
 [834, 8880, 11],
 [853, 8880, 11],
 [854, 888

In [41]:
subject_data.head()

,subject_id,type_of_education_id,no_of_lesson,grade_id
10,832,1,1,11
11,832,2,2,11
12,832,4,1,11
13,832,5,1,11
14,832,3,2,11


In [42]:
subject_list_data.head()

,id,name
0,831,GDTC
1,832,Tin
2,833,GTS-KNS
3,834,Toán
4,835,Văn


In [43]:
#Function for choosing teacher
def getRandomTeacher(subject_list):
        inchargeTeacher = []
        subject_list = json.loads(subject_list)
        for s in subject_list:
            temp = []
            for t in teacher_data_list:
                if t[0] == s: temp.append(t)
            if len(temp) == 0:
                inchargeTeacher.append([s, -1, grade])
            else:
                inchargeTeacher.append(rnd.choice(temp))
        return inchargeTeacher

In [44]:
#Create timtable formation 
timetable = class_data.copy(deep=True)
timetable['teacher'] = timetable['subject_list'].apply(lambda x: getRandomTeacher(x))
timetable = timetable.explode('teacher').drop('subject_list', axis=1)
timetable['subject'] = timetable['teacher'].str[0]

timetable = timetable.merge(subject_data, left_on=['subject', 'type_of_education_id'],
                            right_on=['subject_id', 'type_of_education_id'])
timetable = timetable.merge(format_type_data, left_on='format_type_id', right_on='id')
timetable = pd.DataFrame(timetable.values.repeat(timetable.no_of_lesson, axis=0), columns=timetable.columns).astype(
    timetable.dtypes)
timetable['selected_timeslot'] = ""

for idx, teacher in enumerate(timetable['teacher'].values):
    timetable.at[idx, 'teacher'] = teacher[1]
timetable['teacher'] = pd.to_numeric(timetable['teacher'])
timetable = timetable.drop(['type_of_education_id', 'subject_id', 'no_of_lesson'], axis=1)

timetable = timetable.merge(subject_list_data, left_on='subject', right_on='id').drop('id_y', axis=1)
timetable = timetable.rename(columns={'id_x': 'id'})
timetable = timetable.drop(['grade_id_x', 'grade_id_y'], axis=1)

In [45]:
timetable

,id,class_name,format_type_id,teacher,subject,timeslot,selected_timeslot,name
0,2843,11SN1,1,8898,841,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...",,Anh
1,2843,11SN1,1,8898,841,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...",,Anh
2,2843,11SN1,1,8898,841,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...",,Anh
3,2843,11SN1,1,8898,841,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...",,Anh
4,2843,11SN1,1,8898,841,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...",,Anh
...,...,...,...,...,...,...,...,...
722,2862,11D10,3,8889,835,"[(2, 181, Tiết 1 sáng), (4, 181, Tiết 1 sáng),...",,Văn
723,2862,11D10,3,8889,835,"[(2, 181, Tiết 1 sáng), (4, 181, Tiết 1 sáng),...",,Văn
724,2862,11D10,3,8889,835,"[(2, 181, Tiết 1 sáng), (4, 181, Tiết 1 sáng),...",,Văn
725,2862,11D10,3,8889,835,"[(2, 181, Tiết 1 sáng), (4, 181, Tiết 1 sáng),...",,Văn


In [46]:
#Create cost function
generation = 0
elite = None
population_size = 20
population = []
rankingList = []

def getFitness(dataframe):
    fitness = len(dataframe.id)
    hard, soft = 0, 0
    class_duplicate = pd.DataFrame(
        dataframe.groupby(['class_name', 'selected_timeslot'], as_index=False)['class_name'].agg(
            {'class_count': 'count'}))
    teacher_duplicate = pd.DataFrame(
        dataframe.groupby(['teacher', 'selected_timeslot'], as_index=False)['teacher'].agg(
            {'teacher_count': 'count'}))
    class_duplicate = class_duplicate.loc[(class_duplicate.class_count > 1)]
    teacher_duplicate = teacher_duplicate.loc[(teacher_duplicate.teacher_count > 1)]
    teacher_duplicate = teacher_duplicate.loc[(teacher_duplicate.teacher != -1)]
    hard += len(class_duplicate.class_name) + len(teacher_duplicate.teacher)
    return fitness - hard

In [47]:
for s in range(population_size):
    schedule = copy.deepcopy(timetable)
    schedule = schedule.sort_values(by=['id']).reset_index(drop=True)
    current_class = None
    current_timeslot_list = None
    for idx, row in schedule.iterrows():
        if current_class != row['id']:
            current_class = row['id']
            current_timeslot_list = copy.deepcopy(row['timeslot'])
        if row['name'] == 'SH-CC':
            if row['format_type_id'] == 1 or row['format_type_id'] == 4:
                for item in current_timeslot_list:
                    if item[0] == 1 and item[2] == 'Tiết 1 sáng':
                        schedule.at[idx, 'selected_timeslot'] = copy.deepcopy(item)
                        current_timeslot_list.remove(item)
            elif row['format_type_id'] == 2 or row['format_type_id'] == 3:
                for item in current_timeslot_list:
                    if item[0] == 1 and item[2] == 'Tiết 4 chiều':
                        schedule.at[idx, 'selected_timeslot'] = copy.deepcopy(item)
                        current_timeslot_list.remove(item)
        else:
            random_index = rnd.randrange(0, len(current_timeslot_list))
            if row['format_type_id'] == 1 or row['format_type_id'] == 4:
                while current_timeslot_list[random_index][0] == 1 and current_timeslot_list[random_index][
                    2] == 'Tiết 1 sáng':
                    random_index = rnd.randrange(0, len(current_timeslot_list))
                schedule.at[idx, 'selected_timeslot'] = current_timeslot_list[random_index]
                del current_timeslot_list[random_index]
            if row['format_type_id'] == 2 or row['format_type_id'] == 3:
                while current_timeslot_list[random_index][0] == 1 and current_timeslot_list[random_index][
                    2] == 'Tiết 4 chiều':
                    random_index = rnd.randrange(0, len(current_timeslot_list))
                schedule.at[idx, 'selected_timeslot'] = current_timeslot_list[random_index]
                del current_timeslot_list[random_index]

    population.append(schedule)

for individual in population:
    rankingList.append({'schedule': individual, 'fitness': getFitness(individual)})

rankingList_sorted = sorted(rankingList, key=lambda x: x['fitness'], reverse=True)
elite = rankingList_sorted[0]

while generation < 30:
    generation += 1
    population = []
    rankingList = []
    timetable = class_data.copy(deep=True)
    timetable['teacher'] = timetable['subject_list'].apply(lambda x: getRandomTeacher(x))
    timetable = timetable.explode('teacher').drop('subject_list', axis=1)
    timetable['subject'] = timetable['teacher'].str[0]

    timetable = timetable.merge(subject_data, left_on=['subject', 'type_of_education_id'],
                                right_on=['subject_id', 'type_of_education_id'])
    timetable = timetable.merge(format_type_data, left_on='format_type_id', right_on='id')
    timetable = pd.DataFrame(timetable.values.repeat(timetable.no_of_lesson, axis=0),
                             columns=timetable.columns).astype(
        timetable.dtypes)
    timetable['selected_timeslot'] = ""

    for idx, teacher in enumerate(timetable['teacher'].values):
        timetable.at[idx, 'teacher'] = teacher[1]
    timetable['teacher'] = pd.to_numeric(timetable['teacher'])
    timetable = timetable.drop(['type_of_education_id', 'subject_id', 'no_of_lesson'], axis=1)

    timetable = timetable.merge(subject_list_data, left_on='subject', right_on='id').drop('id_y', axis=1)
    timetable = timetable.rename(columns={'id_x': 'id'})

    for s in range(population_size):
        schedule = copy.deepcopy(timetable)
        schedule = schedule.sort_values(by=['id']).reset_index(drop=True)
        current_class = None
        current_timeslot_list = None
        for idx, row in schedule.iterrows():
            if current_class != row['id']:
                current_class = row['id']
                current_timeslot_list = copy.deepcopy(row['timeslot'])
            if row['name'] == 'SH-CC':
                if row['format_type_id'] == 1 or row['format_type_id'] == 4:
                    for item in current_timeslot_list:
                        if item[0] == 1 and item[2] == 'Tiết 1 sáng':
                            schedule.at[idx, 'selected_timeslot'] = copy.deepcopy(item)
                            current_timeslot_list.remove(item)
                elif row['format_type_id'] == 2 or row['format_type_id'] == 3:
                    for item in current_timeslot_list:
                        if item[0] == 1 and item[2] == 'Tiết 4 chiều':
                            schedule.at[idx, 'selected_timeslot'] = copy.deepcopy(item)
                            current_timeslot_list.remove(item)
            else:
                random_index = rnd.randrange(0, len(current_timeslot_list))
                if row['format_type_id'] == 1 or row['format_type_id'] == 4:
                    while current_timeslot_list[random_index][0] == 1 and current_timeslot_list[random_index][
                        2] == 'Tiết 1 sáng':
                        random_index = rnd.randrange(0, len(current_timeslot_list))
                    schedule.at[idx, 'selected_timeslot'] = current_timeslot_list[random_index]
                    del current_timeslot_list[random_index]
                if row['format_type_id'] == 2 or row['format_type_id'] == 3:
                    while current_timeslot_list[random_index][0] == 1 and current_timeslot_list[random_index][
                        2] == 'Tiết 4 chiều':
                        random_index = rnd.randrange(0, len(current_timeslot_list))
                    schedule.at[idx, 'selected_timeslot'] = current_timeslot_list[random_index]
                    del current_timeslot_list[random_index]
        population.append(schedule)

    for individual in population:
        rankingList.append({'schedule': individual, 'fitness': getFitness(individual)})

    rankingList_sorted = sorted(rankingList, key=lambda x: x['fitness'], reverse=True)
    if rankingList_sorted[0]['fitness'] > elite['fitness']:
        elite = rankingList_sorted[0]

In [48]:
print(f'Best: {elite["fitness"]}')
elite['schedule']

Best: 705


,id,class_name,grade_id_x,format_type_id,teacher,subject,grade_id_y,timeslot,selected_timeslot,name
0,2843,11SN1,11,1,8898,841,11,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...","(1, 182, Tiết 2 sáng)",Anh
1,2843,11SN1,11,1,8922,844,11,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...","(4, 182, Tiết 2 sáng)",Lý
2,2843,11SN1,11,1,8878,834,11,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...","(1, 183, Tiết 3 sáng)",Toán
3,2843,11SN1,11,1,8892,835,11,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...","(2, 182, Tiết 2 sáng)",Văn
4,2843,11SN1,11,1,8892,835,11,"[(1, 181, Tiết 1 sáng), (2, 181, Tiết 1 sáng),...","(6, 181, Tiết 1 sáng)",Văn
...,...,...,...,...,...,...,...,...,...,...
722,2862,11D10,11,3,8904,841,11,"[(2, 181, Tiết 1 sáng), (4, 181, Tiết 1 sáng),...","(5, 186, Tiết 1 chiều)",Anh
723,2862,11D10,11,3,8904,841,11,"[(2, 181, Tiết 1 sáng), (4, 181, Tiết 1 sáng),...","(2, 186, Tiết 1 chiều)",Anh
724,2862,11D10,11,3,8904,841,11,"[(2, 181, Tiết 1 sáng), (4, 181, Tiết 1 sáng),...","(1, 190, Tiết 5 chiều)",Anh
725,2862,11D10,11,3,8904,841,11,"[(2, 181, Tiết 1 sáng), (4, 181, Tiết 1 sáng),...","(6, 184, Tiết 4 sáng)",Anh
